# Task 1

Da det i Assignment 1 gav bedst mening at bruge ``pandas``, da filen er en csv som passer godt til en dataframe, og da ``pandas`` kommer med en nem måde at læse csv filen på, har jeg valgt her igen at bruge ``pandas`` til at læse og arbejde ned [``news_sample.csv``](https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv).

In [1]:
import pandas as pd

DATA_FILE = "data.csv"
news_sample = pd.read_csv(DATA_FILE)
#news_sample = pd.read_csv('https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv').drop(columns=['Unnamed: 0'])
news_sample.head()


,type,title,cleaned
0,political,Iran News Round Up,"['plu', 'one', 'articl', 'googl', 'plu', 'than..."
1,fake,The Cost Of The Best Senate Banking Committee ...,"['cost', 'best', 'senat', 'bank', 'committe', ..."
2,satire,Man Awoken From 27-Year Coma Commits Suicide A...,"['man', 'awoken', '<num>', 'year', 'coma', 'co..."
3,reliable,Opening a Gateway for Girls to Enter the Compu...,"['julia', 'geist', 'ask', 'draw', 'pictur', 'c..."
4,conspiracy,100 Compiled Studies on Vaccine Dangers – Infi...,"['<num>', 'compil', 'studi', 'vaccin', 'danger..."


Opgaven er her ligesom i Assignment 1, at "structure, process, clean it". Jeg er ikke sikker på hvad structure og process refere til, så jeg har bare tænkt mig at cleane det.

I cleaning processen fra Assignment 1 var opgaven at:
- Lowercase teksten
- Udskifte tal med ``<NUM>``
- Udskifte datoer med ``<DATE>``
- Udskifte emails med ``<EMAIL>``
- Udskifte URLs med ``<URL>``
- Fjerne ekstra white-space

Python biblioteket ``cleantext`` har indbygget funktionalitet for mange af disse, men jeg er utilfreds med måden den udskifter tal på, samt så mangler den en måde at udskifte datoer på, så derfor vil jeg kun gøre delvist brug af dette bibliotekt. Dog har den også yderligere cleaning funktionaliteter som jeg vil gøre brug af.

Den anbefalede rækkefølge for cleaning processen er:
1. Lowercase teksten
2. Udskifte URL med ``<URL>``
3. Udskifte email med ``<EMAIL>``
4. Udskifte tlf.nr. med ``<PHONE>``
5. Udskifte datoer med ``<DATE>``
6. Udskifte valuta symboler med ``<CUR>``
7. Udfskifte tal med ``<NUM>``
8. Fjerne ekstra white-space

Dog vil jeg kun delvist gøre brug af denne rækkefølge dan ``cleantext`` kan gøre flere af disse på én gang.

In [2]:
import cleantext as clean
import re

# Converting to lowercase
def lowercase(text:str):
    return text.lower()

url_regex = re.compile(r'(?:http[s]?://)?(?:www\.)?[\w]+\.[a-z]{2,}[\w#-_]*')
# Replacing URLs
def sub_URL(text:str):
    return url_regex.sub(' <URL> ', text)

email_regex = re.compile(r'[\w._-]+@[\w._-]+\.[a-z]{2,}')
# Replacing Emails
def sub_EMAIL(text:str):
    return email_regex.sub(' <EMAIL> ', text)

date1_regex = re.compile(r'(?:\b\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}\b)') # matcher 20-02-2025 og 20/02/2025 og 20.02.2025 og 2-20-2025 og 2/20/2025 og 2025-02-20 og 20/2/25 2/20/25
date2_regex = re.compile(r'(?:(?:(?:the )?(?:[123]\d|\d)(?:st|nd|rd|th)? (?:of )?)?(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|sep|dec|mon|tue|wed|thu|fri|sat|sun)[a-z]*\.?(?: (?:the )?\d{1,4}(?:st|nd|rd|th)?,?))(?: (?:20|19)[\d]{2}s?\b)?')
date3_regex = re.compile(r'(?:(?:(?:the )?(?:[123]\d|\d)(?:st|nd|rd|th)? of )(?:jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|sep|dec|mon|tue|wed|thu|fri|sat|sun)[a-z]*)')
date4_regex = re.compile(r'(?:20|19)[\d]{2}s?\b')
# Replacing Dates
def sub_DATE(text:str):
    return date4_regex.sub(' <DATE> ', date3_regex.sub(' <DATE> ', date2_regex.sub(' <DATE> ', date1_regex.sub(' <DATE> ', text))))

num_regex = re.compile(r'\d(?:\d|\.|\,|st|nd|rd|th)*')
# Replacing Numbers
def sub_NUM(text:str):
    return num_regex.sub(' <NUM> ', text)

punct_regex = re.compile(r'[^\w\s](?=[a-z])')
# Replacing Punctuation in words
def sub_punct(text:str):
    return punct_regex.sub(' ', text)

# rep URL, rep Email, rep tlf.nr., rep Numbers, del punct, rep Valuta
def cleaning_module(text:str):
    return clean.normalize_whitespace(clean.remove_punct(clean.replace_currency_symbols(text)))

Jeg samler funktionerne der udføre hver opgave seperat, i én funktion ``CleanText``, hvor hvert cleaning step udføres i det korrekte rækkefølge. Nedenfor køre funktionerne tidligere desto mere indekseret det er.

In [3]:
def CleanText(text:str) -> str :
    """Returns a cleaned string.
    
    Args:
        text (str) : A (raw) uncleaned text (document).
    Returns:
        str : The same text, but cleaned."""
    #hvis teksten indholder NaNa, retuner ""
    if pd.isna(text):
        return ""
    return cleaning_module(
        sub_punct(
            sub_NUM(
                sub_DATE(
                    sub_EMAIL(
                        sub_URL(
                            lowercase(
                                text
                            )
                        )
                    )
                )
            )
        )
    )

# Test
CleanText(r'Sec. 3. Coordination. (a) The Director of the Office of Science and Technology Policy (OSTP), in consultation with the Assistant to the President for Homeland Security and Counterterrorism and the Director of the Office of Management and Budget (OMB), shall coordinate the development and implementation of Federal Government activities to prepare the Nation for space weather events, including the activities established in section 5 of this order and the recommendations of the National Science and Technology Council (NSTC), established by Executive Order 12881 of November 23, 1993 (Establishment of the National Science and Technology Council).')


'sec <NUM> coordination a the director of the office of science and technology policy ostp in consultation with the assistant to the president for homeland security and counterterrorism and the director of the office of management and budget omb shall coordinate the development and implementation of federal government activities to prepare the nation for space weather events including the activities established in section <NUM> of this order and the recommendations of the national science and technology council nstc established by executive order <NUM> <DATE> establishment of the national science and technology council'

Funktionen ``CleanText`` operere på én string ad gangen, så dette betyder at den skal påføres (apply) på hver dokument (string/data point) i vores datasæt. Altså skal du bruge noget der iterere over hvert element i datasættet, eksempelvis et ``for`` loop eller ``<DATA FRAME>.apply(<FUNKTION>)`` fra panda biblioteket.

Definere en ny dataframe for den ikke-cleanede og det cleanede data, for nemmere databehandling.

In [5]:
# Defining dataframe for data manipulation
#sample_documents = news_sample[['content']].rename(columns={'content': 'Original'})
sample_documents = news_sample[['cleaned']].rename(columns={'cleaned': 'Original'})

# Adding cleaned column
sample_documents['Cleaned'] = sample_documents['Original'].apply(CleanText)

sample_documents



,Original,Cleaned
0,"['plu', 'one', 'articl', 'googl', 'plu', 'than...",plu one articl googl plu thank ali alfoneh ass...
1,"['cost', 'best', 'senat', 'bank', 'committe', ...",cost best senat bank committe jp morgan buy cu...
2,"['man', 'awoken', '<num>', 'year', 'coma', 'co...",man awoken num> year coma commit suicid learn ...
3,"['julia', 'geist', 'ask', 'draw', 'pictur', 'c...",julia geist ask draw pictur comput scientist l...
4,"['<num>', 'compil', 'studi', 'vaccin', 'danger...",num> compil studi vaccin danger activist post ...
...,...,...
994983,"['<num>', 'wire', '+', 'intervent', 'watch', '...",num> wire + intervent watch | say anyon sign a...
994984,"['plu', 'one', 'articl', 'googl', 'plu', 'than...",plu one articl googl plu thank ali alfoneh ass...
994985,"['china', 'russia', 'acquir', 'gold', 'dump', ...",china russia acquir gold dump us dollar evid c...
994986,"['liverpool', 'sign', 'sadio', 'mane', 'southa...",liverpool sign sadio mane southampton fan reac...


Beregner størrelsen af ordforrådet (antal unikke ord), ved brug at set-oprators. Jeg beregner det samlede ordforråd for alle dokumenter i datasættet. Først beregner jeg ordforrådet for det cleanede dataset med stopwords, hvorefter jeg laver et nyt sæt, der er sættet af alle ord i ``vocab_with_stopwords`` uden alle ord i sættet ``english_stopwords``.

In [6]:
# Combined vocab of all documents
vocab_with_stopwords = set()

# Get combined vocab from strings
def get_vocab(tokens:list[str], vocab:set[str]):
    vocab.update(set(tokens))

# Calculating the vocab
sample_documents['Cleaned'].apply(lambda x: get_vocab(x.split(), vocab_with_stopwords))


# Importing stopwords
from nltk.corpus import stopwords
english_stopwords = set(stopwords.words('english'))

# Calculating the vocab without stopwords
vocab_without_stopwords = vocab_with_stopwords - english_stopwords

print(vocab_with_stopwords.__len__(), vocab_without_stopwords.__len__(), english_stopwords.__len__())

1127553 1127412 198


Effekten af at fjerne stopwords har en mindre indflydelse på ordforrådets længde end forventet, men dette giver mening da antallet af unikke stopord ikke er stor, og da ``vocab_with_stopwords`` er et sæt, altså hvert ord optræder kun én gang, kan der ikke fjernes flere ord fra ordforrådet end sættet ``english_stopwords`` er lang.

Da vi skal beregne størrelsen af ordforrådet før og efter *stemming* antager vi at dette betyder at vi stopwords allerede er fjernet. Vi bruger ``PorterStemmer().stem`` fra ``nltk.stem.porter``, til at lave en nyt ordforråd (set) af ord der har fået påført *stemming*.

In [19]:
from nltk.stem.porter import PorterStemmer

# Calculating the vocab without stopwords
vocab_with_stemming = {PorterStemmer().stem(word) for word in vocab_without_stopwords}

print(vocab_with_stemming.__len__())

1115465


Dette program opdeler et renset nyhedsdata-sæt i 80% træning, 10% validering og 10% test ved hjælp af StratifiedShuffleSplit. Det sikrer, at fordelingen af nyhedstyper bevares i alle datasæt, hvilket forbedrer modelens præcision.

In [8]:
# Koden køres på rensede data, derfor skal tidligere opgaver være kørt for at denne del kan fungere.
# Det kræver, at pakken installeres med: "pip install scikit-learn"

from sklearn.model_selection import StratifiedShuffleSplit

#fejner NaNa i texten, fordi det give fejl, når skulle dele opgave toi træning og test modellen
news_sample = news_sample.dropna(subset=['cleaned', 'type'])
# Bekræft at NaN er fjernet
print("Antal NaN efter fjernelse:", news_sample.isna().sum().sum())


# Vi bruge trining modeken på cleand data 
X= news_sample ['cleaned']
y= news_sample ['type'] #Brug nyhedstype som labels
#vi er nyskerige om, hvor mange % af hver nyhedstype
#print(y.value_counts(normalize=True))

# Opret StratifiedShuffleSplit objekt
split= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, temp_index in split.split(X, y):
    X_train, X_temp = X.iloc[train_index], X.iloc[temp_index]
    y_train, y_temp = y.iloc[train_index], y.iloc[temp_index]
    

# Størrelsen på splits
print(f"Dataerne der brugt for triaaning, som svar til 80: {len(X_train)}")
print(f"Dataerne der blive brugt for (validering+ test): {len(X_temp)} rækker")




# Opret StratifiedShuffleSplit objekt
split= StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
for val_index, test_index in split.split(X_temp, y_temp):
    X_val, X_test = X_temp.iloc[val_index], X_temp.iloc[test_index]
    y_val, y_test = y_temp.iloc[val_index], y_temp.iloc[test_index]

# Størrelsen på splits
print(f"Dataerne der brugt for validering (10%): {len(X_val)} rækker")
print(f"Dataerne der blive brugt for test (10%): {len(X_test)} rækker")

Antal NaN efter fjernelse: 8580
Dataerne der brugt for triaaning, som svar til 80: 757769
Dataerne der blive brugt for (validering+ test): 189443 rækker
Dataerne der brugt for validering (10%): 94721 rækker
Dataerne der blive brugt for test (10%): 94722 rækker


In [9]:
!git status




Python(31357) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


On branch newsproj
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Text Cleaning (1).ipynb
	Text Cleaning.ipynb
	data.csv
	myenv/

nothing added to commit but untracked files present (use "git add" to track)
